In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\bandana\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [4]:
#Dependencies

import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [5]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [6]:
df.koi_disposition.value_counts()

FALSE POSITIVE    3504
CONFIRMED         1800
CANDIDATE         1687
Name: koi_disposition, dtype: int64

In [7]:
# look at confirmed as true and false positives as false (1 and 0) and remove all candidate planets.
df = df.loc[df["koi_disposition"] != "CANDIDATE"]

df.koi_disposition.value_counts()

FALSE POSITIVE    3504
CONFIRMED         1800
Name: koi_disposition, dtype: int64

In [8]:
df['koi_disposition'] = df["koi_disposition"].apply(lambda x: 1 if(x=="CONFIRMED") else 0)
y = df['koi_disposition']
df.koi_disposition.value_counts()


0    3504
1    1800
Name: koi_disposition, dtype: int64

In [9]:
# There are three different categories the planets can be classified.
# Let's start with all features and then remove features that have little weight.

#  Avoid linear regression, this is a classification problem.
# Start with logistic classification.
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

# Select your features (columns)

In [10]:
# Set features. This will also be used as your x values.
#selected_features = df[['names', 'of', 'selected', 'features', 'here']]

selected_features = df[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr',
                        'koi_tce_plnt_num', 'koi_steff', 'koi_slogg', 'koi_srad', 'ra', 'dec', 'koi_kepmag']]


# Create a Train Test Split

Use `koi_disposition` for the y values

In [11]:
from sklearn.model_selection import train_test_split

X = selected_features
y = df.koi_disposition

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=42)

In [12]:
X_train.head()

,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
6742,235.206130,291.08930,0.2970,6.989,642.0,1.71,230,0.66,6.8,1,5205,4.648,0.679,288.43362,41.358391,15.917
3453,3.000119,132.90970,0.6350,6.610,7.7,0.73,2566,10247.93,7.3,1,8431,3.970,2.485,292.91702,40.092430,11.960
674,4.487588,133.36092,0.5320,2.874,112.0,1.37,1305,684.07,25.9,1,6060,4.227,1.257,284.86014,39.240551,13.446
5211,98.102098,170.33189,0.1865,1.225,184.4,1.97,525,17.96,6.9,2,6669,4.224,1.427,288.72083,42.716980,13.557
1786,44.616751,164.45238,0.6180,7.831,239.7,2.31,693,54.56,28.4,1,6763,4.220,1.424,292.47797,46.022831,13.791


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [16]:
# Scale your data

# Setup MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# Set initial sizing of minmaxscaler using training data
X_scaler = MinMaxScaler().fit(X_train)

# As y is just 1 or zero, so no need to scale it!

# Use MinMaxScaler to transform the data in the training and test data sets
# So that it is not weighted improperly

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# y_train_scaled = y_scaler.transform(y_train)
# y_test_scaled = y_scaler.transform(y_test)

# Train the Model



In [28]:
#first, use logistic regression in this notebook!
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
#create the model using the training data!
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [30]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.7841150129625265
Testing Data Score: 0.7851083883129123


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [31]:
# Create the GridSearchCV model

from sklearn.model_selection import GridSearchCV

param_grid = {'C': [1, 5, 10, 50, 100, 200, 500],
              'max_iter': [100, 200, 300],
              'tol': [.0001, 0.001, 0.005]}

grid = GridSearchCV(model, param_grid, verbose=3)

In [32]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)


Fitting 5 folds for each of 63 candidates, totalling 315 fits
[CV] C=1, max_iter=100, tol=0.0001 ...................................
[CV] ....... C=1, max_iter=100, tol=0.0001, score=0.763, total=   0.2s
[CV] C=1, max_iter=100, tol=0.0001 ...................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] ....... C=1, max_iter=100, tol=0.0001, score=0.771, total=   0.1s
[CV] C=1, max_iter=100, tol=0.0001 ...................................
[CV] ....... C=1, max_iter=100, tol=0.0001, score=0.793, total=   0.1s
[CV] C=1, max_iter=100, tol=0.0001 ...................................
[CV] ....... C=1, max_iter=100, tol=0.0001, score=0.797, total=   0.1s

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s



[CV] C=1, max_iter=100, tol=0.0001 ...................................
[CV] ....... C=1, max_iter=100, tol=0.0001, score=0.775, total=   0.1s
[CV] C=1, max_iter=100, tol=0.001 ....................................
[CV] ........ C=1, max_iter=100, tol=0.001, score=0.763, total=   0.0s
[CV] C=1, max_iter=100, tol=0.001 ....................................
[CV] ........ C=1, max_iter=100, tol=0.001, score=0.771, total=   0.1s
[CV] C=1, max_iter=100, tol=0.001 ....................................
[CV] ........ C=1, max_iter=100, tol=0.001, score=0.793, total=   0.1s
[CV] C=1, max_iter=100, tol=0.001 ....................................
[CV] ........ C=1, max_iter=100, tol=0.001, score=0.797, total=   0.0s
[CV] C=1, max_iter=100, tol=0.001 ....................................
[CV] ........ C=1, max_iter=100, tol=0.001, score=0.775, total=   0.0s
[CV] C=1, max_iter=100, tol=0.005 ....................................
[CV] ........ C=1, max_iter=100, tol=0.005, score=0.763, total=   0.1s
[CV] 

...

[CV] ...... C=500, max_iter=300, tol=0.001, score=0.866, total=   0.3s
[CV] C=500, max_iter=300, tol=0.001 ..................................
[CV] ...... C=500, max_iter=300, tol=0.001, score=0.848, total=   0.3s
[CV] C=500, max_iter=300, tol=0.005 ..................................
[CV] ...... C=500, max_iter=300, tol=0.005, score=0.840, total=   0.2s
[CV] C=500, max_iter=300, tol=0.005 ..................................
[CV] ...... C=500, max_iter=300, tol=0.005, score=0.846, total=   0.3s
[CV] C=500, max_iter=300, tol=0.005 ..................................
[CV] ...... C=500, max_iter=300, tol=0.005, score=0.855, total=   0.3s
[CV] C=500, max_iter=300, tol=0.005 ..................................


C:\Users\BANDANA\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[CV] ...... C=500, max_iter=300, tol=0.005, score=0.866, total=   0.3s
[CV] C=500, max_iter=300, tol=0.005 ..................................
[CV] ...... C=500, max_iter=300, tol=0.005, score=0.848, total=   0.4s


[Parallel(n_jobs=1)]: Done 315 out of 315 | elapsed:  1.0min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 5, 10, 50, 100, 200, 500],
                         'max_iter': [100, 200, 300],
                         'tol': [0.0001, 0.001, 0.005]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [33]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 500, 'max_iter': 300, 'tol': 0.0001}
0.8508158877258483


In [34]:
#compare with the test data!
predictions = grid.predict(X_test_scaled)
print(sum(predictions), sum(y_test))

total = 0

for i in range(len(predictions)):
    if(predictions[i] == y_test.array[i]):
        total +=1 
        
print(total/len(predictions))

370 343
0.8369462770970783


In [35]:
#check classification report for how the tuned model did!
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions,
                            target_names=["Confirmed", "False Positive"]))

                precision    recall  f1-score   support

     Confirmed       0.89      0.86      0.88       718
False Positive       0.73      0.79      0.76       343

      accuracy                           0.84      1061
     macro avg       0.81      0.82      0.82      1061
  weighted avg       0.84      0.84      0.84      1061



# Save the Model

In [37]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'Bandana_model.sav'
joblib.dump(grid, filename)

['Bandana_model.sav']